In [1]:
import numpy as _numpy_

from controllables.core.tools.gymnasium import (
    BoxSpace,
    DictSpace,
)
from controllables.core.tools.rllib import (
    Env,
)
from controllables.energyplus import System
import ray
from ray import tune , air
from ray.tune.schedulers import PopulationBasedTraining

from controllables.core import Variable, BaseVariable
from controllables.core.callbacks import CallbackManager
from controllables.energyplus import (
    Actuator,
    OutputVariable,
)
import pythermalcomfort as pytc
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.impala import ImpalaConfig


from ray.rllib.algorithms.callbacks import DefaultCallbacks

import logging
import csv
import math

#logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', filename='training.log')


class RewardFunction:
    def __init__(self, metab_rate=1.5, clothing=.5, pmv_limit=.5):
        self._metab_rate = _numpy_.asarray(metab_rate)
        self._clothing = _numpy_.asarray(clothing)
        self._pmv_limit = _numpy_.asarray(pmv_limit)
    
   
    def __call__(self, agent):
        total_reward = 0
        
        for zone in [
            '1FFIRSTFLOORWEST:OPENOFFICE',
            '1FFIRSTFLOOREAST:OPENOFFICE',
            '0FGROUNDFLOORWEST:OPENOFFICE',
            '0FGROUNDFLOOREAST:OPENOFFICE',
            '1FFIRSTFLOORWEST1:OPENOFFICE',
            '1FFIRSTFLOOREAST1:OPENOFFICE',
            '0FGROUNDFLOORWEST1:OPENOFFICE',
            '0FGROUNDFLOOREAST1:OPENOFFICE'
        ]:
            observation = agent.observation.value     
            AHU_COOLING_COIL = observation['AHU COOLING COIL']
            Office_Occupancy = observation['Office Occupancy']
            pmv = pytc.models.pmv_ppd(
                tdb=(tdb := observation[f'{zone} temperature:drybulb']), 
                tr=observation[f'{zone} temperature:radiant'], 
                vr=pytc.utilities.v_relative(v=observation.get('airspeed', .1), met=self._metab_rate), 
                rh=observation[f'{zone} humidity'], 
                met=self._metab_rate, 
                clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
                limit_inputs=False,
            )['pmv']

            penalty = 0
            if Office_Occupancy != 0:  
                if _numpy_.abs(pmv) > self._pmv_limit:
                    penalty = 10 * (_numpy_.abs(pmv) - self._pmv_limit)
                else:
                    penalty = 0

            log_term = max(2 * _numpy_.abs(pmv), 1e-2) 
            
            reward = (
                2 * Office_Occupancy * (-_numpy_.log(log_term))  
                - penalty
            )
            reward = min(reward, 10)
            
            total_reward += reward
        total_reward = total_reward - (_numpy_.exp(AHU_COOLING_COIL / 180000) - 1)
        
        return total_reward

class CustomExternalEnv(Env):
    action_space = DictSpace({
        f'thermostat_{zone}': BoxSpace(
            low=20., high=30.,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(Actuator.Ref(
            type='Schedule:Compact',
            control_type='Schedule Value',
            key=f'{zone} COOLING SETPOINT SCHEDULE',
        ))
        for zone in [
            '1FFIRSTFLOORWEST:OPENOFFICE',
            '1FFIRSTFLOOREAST:OPENOFFICE',
            '0FGROUNDFLOORWEST:OPENOFFICE',
            '0FGROUNDFLOOREAST:OPENOFFICE',
            '1FFIRSTFLOORWEST1:OPENOFFICE',
            '1FFIRSTFLOOREAST1:OPENOFFICE',
            '0FGROUNDFLOORWEST1:OPENOFFICE',
            '0FGROUNDFLOOREAST1:OPENOFFICE'
        ]
    })    

    observation_space_dict = dict({
        'AHU COOLING COIL': BoxSpace(
            low=-_numpy_.inf, high=+_numpy_.inf,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(OutputVariable.Ref(
            type='Cooling Coil Total Cooling Rate',
            key='AIR LOOP AHU COOLING COIL',
        )),
        'Office Occupancy': BoxSpace(
            low=-_numpy_.inf, high=+_numpy_.inf,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(OutputVariable.Ref(
            type='Schedule Value',
            key='Office_OpenOff_Occ',
        )),
    })

    for zone in [
        '1FFIRSTFLOORWEST:OPENOFFICE',
        '1FFIRSTFLOOREAST:OPENOFFICE',
        '0FGROUNDFLOORWEST:OPENOFFICE',
        '0FGROUNDFLOOREAST:OPENOFFICE',
        '1FFIRSTFLOORWEST1:OPENOFFICE',
        '1FFIRSTFLOOREAST1:OPENOFFICE',
        '0FGROUNDFLOORWEST1:OPENOFFICE',
        '0FGROUNDFLOOREAST1:OPENOFFICE'
    ]:
        observation_space_dict.update({
            f'{zone} temperature:drybulb': BoxSpace(
                low=-_numpy_.inf, high=+_numpy_.inf,
                dtype=_numpy_.float32,
                shape=(),
            ).bind(OutputVariable.Ref(
                type='Zone Mean Air Temperature',
                key=f'{zone}',
            )),
            f'{zone} temperature:radiant': BoxSpace(
                low=-_numpy_.inf, high=+_numpy_.inf,
                dtype=_numpy_.float32,
                shape=(),
            ).bind(OutputVariable.Ref(
                type='Zone Mean Radiant Temperature',
                key=f'{zone}',
            )),
            f'{zone} humidity': BoxSpace(
                low=-_numpy_.inf, high=+_numpy_.inf,
                dtype=_numpy_.float32,
                shape=(),
            ).bind(OutputVariable.Ref(
                type='Zone Air Relative Humidity',
                key=f'{zone}',
            ))            
        })
    
    observation_space = DictSpace(observation_space_dict)
    
    reward = RewardFunction()

    class PMVVariable(BaseVariable):
        def __init__(
            self, 
            agent,
            metab_rate=1.5, clothing=.5, pmv_limit=.5,
        ):
            self._agent = agent
            self._metab_rate = _numpy_.asarray(metab_rate)
            self._clothing = _numpy_.asarray(clothing)
            self._pmv_limit = _numpy_.asarray(pmv_limit)
        
        @property
        def value(self):
            observation = self._agent.observation.value

            res = pytc.models.pmv_ppd(
                tdb=(tdb := observation['temperature:drybulb']), 
                tr=observation['temperature:radiant'], 
                # calculate relative air speed
                vr=pytc.utilities.v_relative(v=observation.get('airspeed', .1), met=self._metab_rate), 
                rh=observation['humidity'], 
                met=self._metab_rate, 
                # calculate dynamic clothing
                clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
                limit_inputs=False ,
            )['pmv']

            return res

    @property
    def pmv(self):
        return self.PMVVariable(agent=self.agent)

    def __init__(self, config: dict = dict()):
        super().__init__({
            'action_space': self.__class__.action_space,
            'observation_space': self.__class__.observation_space,
            'reward': self.__class__.reward,
            **config,
        })

    def run(self):
        system = System(
            building='all_room_have_hvac.idf',
            weather='SGP_Singapore_486980_IWEC.epw',
            # TODO
            report='tmp/',
            repeat=True,
        )
        #system.add('logging:progress')
        self.__attach__(system).schedule_episode()
        system.start().wait()


E0000 00:00:1730276437.087622 3373898 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730276437.091029 3373898 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered




Failed to import TF-Keras. Please note that TF-Keras is not installed by default when you install TensorFlow Probability. This is so that JAX-only users do not have to install TensorFlow or TF-Keras. To use TensorFlow Probability with TensorFlow, please install the tf-keras or tf-keras-nightly package.
This can be be done through installing the tensorflow-probability[tf] extra.




Failed to import TF-Keras. Please note that TF-Keras is not installed by default when you install TensorFlow Probability. This is so that JAX-only users do not have to install TensorFlow or TF-Keras. To use TensorFlow Probability with TensorFlow, please install the tf-keras or tf-keras-nightly package.
This can be be done through installing the tensorflow-probability[tf] extra.




Failed to import TF-Keras. Please note that TF-Keras is not installed by default when you install TensorFlow Probability. This is so that JAX-only users do not have to install TensorFlow or TF-Keras. To use TensorFlow Probabilit

/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/tensorflow_probability/python/__init__.py:58: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  distutils.version.LooseVersion(required_tensorflow_version)):
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/tensorflow_probability/python/__init__.py:58: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version

In [2]:
from ray.rllib.algorithms.ppo import PPO, PPOConfig

def get_config(): 
    return (
        PPOConfig()
        .environment(CustomExternalEnv)
        .env_runners(
            # NOTE this env (an `ExternalEnv`) does not support connectors
            enable_connectors=False,
            # disable distributed workers for local plotting
            # num_rollout_workers=0,
            # create_env_on_local_worker=True,
        )
        # TODO
        .resources(num_gpus=0)
    )


In [3]:
pbt = PopulationBasedTraining(
    time_attr="training_iteration",
    perturbation_interval=5,  # Number of iterations between hyperparameter adjustments
    resample_probability=0.25,  # Probability of resampling a hyperparameter
    hyperparam_mutations={
        "lr": tune.uniform(1e-5, 0.1),
        "train_batch_size": [4000],
        "sgd_minibatch_size": [32, 64, 128, 256, 512],
        "num_sgd_iter": [10, 20, 30],
        "clip_param": tune.uniform(0.1, 0.3),
        # Add more hyperparameters if needed
    }
)

tuner = tune.Tuner(
    "PPO",
    param_space=get_config().to_dict(),
    tune_config=tune.TuneConfig(
        scheduler=pbt,
        num_samples=4,  # 群体大小
        metric="env_runners/episode_reward_mean",  # 同样的指标
        mode="max",  # 同样的优化方向
    ),
    run_config=air.RunConfig(
        #stop={"env_runners/num_episodes": 100},  # 训练停止条件
        stop={"training_iteration": 100},  # 训练停止条件
        checkpoint_config=air.CheckpointConfig(
        checkpoint_at_end=True  # 在训练结束时保存检查点
        ),
        verbose=2,
    )
)


In [4]:
results = tuner.fit()
best_result = results.get_best_result()
print("Best Hyperparameters found: ", best_result)

/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1362: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-10-30_06-17-03_685256_3364715/logs/gcs_server.out' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1362: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-10-30_06-17-03_685256_3364715/logs/gcs_server.err' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1367: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-10-30_06-17-03_685256_3364715/logs/monitor.out' mode='a' encoding='utf-8'>
  self.start_monitor()
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1367: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session

(pid=3366438) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=3366438) E0000 00:00:1730269025.703128 3366438 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=3366438) E0000 00:00:1730269025.706537 3366438 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(pid=3366437) 
(pid=3366437) 
(pid=3366437) Failed to import TF-Keras. Please note that TF-Keras is not installed by default when you install TensorFlow Probability. This is so that JAX-only users do not have to install TensorFlow or TF-Keras. To use TensorFlow Probability with TensorFlow, please install the tf-keras or tf-keras-nightly package.
(pid=3366437) This can be be done through installing the tensorflow-probability[tf] extra.
(pid=3366437) 
(pid=3366437) 
(pid=3366438) 
(pid=3366438) 
(pid=3366438) 
(pid=3366438) 
(pid=3366440) 
(pid=3366440) 
(pid=3366440) 
(pid=3366440) 
(pid=3366439) 
(pid=3366439) 
(pid=3366439) 
(pid=3366439) 
(pid=3366437) 
(pid=3366437) 
(pid=3366437) 
(pid=3366437) 
(pid=3366437) 
(pid=3366437) 
(pid=3366437) 
(pid=3366437) 
(pid=3366437) 
(pid=3366437) 
(pid=3366437) 
(pid=3366437) 
(pid=3366438) 
(pid=3366438) 
(pid=3366438) 
(pid=3366438) 
(pid=3366438) 
(pid=3366438) 
(pid=3366438) 
(pid=3366438) 
(pid=3366438) 
(pid=3366438) 
(pid=3366438) 
(pid=3

(pid=3366804) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(pid=3366804) E0000 00:00:1730269032.418118 3366804 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered [repeated 4x across cluster]
(pid=3366804) E0000 00:00:1730269032.421669 3366804 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered [repeated 4x across cluster]


(pid=3366805) 
(pid=3366805) 
(pid=3366805) 
(pid=3366805) 
(pid=3366805) Failed to import TF-Keras. Please note that TF-Keras is not installed by default when you install TensorFlow Probability. This is so that JAX-only users do not have to install TensorFlow or TF-Keras. To use TensorFlow Probability with TensorFlow, please install the tf-keras or tf-keras-nightly package. [repeated 16x across cluster]
(pid=3366805) This can be be done through installing the tensorflow-probability[tf] extra. [repeated 16x across cluster]
(pid=3366804) 
(pid=3366804) 
(pid=3366804) 
(pid=3366804) 
(pid=3366811) 
(pid=3366811) 
(pid=3366811) 
(pid=3366811) 
(pid=3366809) 
(pid=3366809) 
(pid=3366809) 
(pid=3366809) 
(pid=3366812) 
(pid=3366812) 
(pid=3366812) 
(pid=3366812) 
(pid=3366808) 
(pid=3366808) 
(pid=3366808) 
(pid=3366808) 
(pid=3366813) 
(pid=3366813) 
(pid=3366813) 
(pid=3366813) 
(pid=3366810) 
(pid=3366810) 
(pid=3366810) 
(pid=3366810) 


...

(pid=3373333) 
(pid=3373333) 
(pid=3373333) 
(pid=3373333) 
(pid=3373333) Failed to import TF-Keras. Please note that TF-Keras is not installed by default when you install TensorFlow Probability. This is so that JAX-only users do not have to install TensorFlow or TF-Keras. To use TensorFlow Probability with TensorFlow, please install the tf-keras or tf-keras-nightly package. [repeated 4x across cluster]
(pid=3373333) This can be be done through installing the tensorflow-probability[tf] extra. [repeated 4x across cluster]
(pid=3373334) 
(pid=3373334) 
(pid=3373334) 
(pid=3373334) 


(RolloutWorker pid=3373334) /home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(RolloutWorker pid=3373334)   if (distutils.version.LooseVersion(tf.__version__) <
(RolloutWorker pid=3373334) /home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/tensorflow_probability/python/__init__.py:58: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(RolloutWorker pid=3373334)   distutils.version.LooseVersion(required_tensorflow_version)):
(RolloutWorker pid=3373334) /home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(RolloutWorker pid=3373334)   if (distutils.version.LooseVersion(tf.__version__) <
(RolloutWorker pid=3373334) /ho

(RolloutWorker pid=3373333) 
(RolloutWorker pid=3373333) 
(RolloutWorker pid=3373333) 
(RolloutWorker pid=3373333) 
(RolloutWorker pid=3373333) 
(RolloutWorker pid=3373333) 
(RolloutWorker pid=3373333) 
(RolloutWorker pid=3373333) 
(RolloutWorker pid=3373333) 
(RolloutWorker pid=3373333) 
(RolloutWorker pid=3373333) 
(RolloutWorker pid=3373333) 
(RolloutWorker pid=3373334) 
(RolloutWorker pid=3373334) 
(RolloutWorker pid=3373334) 
(RolloutWorker pid=3373334) 
(RolloutWorker pid=3373334) 
(RolloutWorker pid=3373334) 
(RolloutWorker pid=3373334) 
(RolloutWorker pid=3373334) 
(RolloutWorker pid=3373334) 
(RolloutWorker pid=3373334) 
(RolloutWorker pid=3373334) 
(RolloutWorker pid=3373334) 


(PPO pid=3373273) Install gputil for GPU system monitoring.
(PPO pid=3373273) 2024-10-30 07:00:22,171	WARNING deprecation.py:50 -- DeprecationWarning: `_enable_new_api_stack` has been deprecated. Use `AlgorithmConfig._enable_new_api_stack` instead. This will raise an error in the future!
(PPO pid=3373273) Restored on 192.168.200.249 from checkpoint: Checkpoint(filesystem=local, path=/home/AD/user/ray_results/PPO_2024-10-30_06-17-02/PPO_CustomExternalEnv_956b5_00002_2_2024-10-30_06-17-04/checkpoint_000004)
(RolloutWorker pid=3373334) /home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/variables.py:345: RuntimeWarning: OutputVariable(OutputVariable.Ref(type='Zone Mean Radiant Temperature', key='1FFIRSTFLOOREAST1:OPENOFFICE')) requested while Kernel() is running; It may not be available until the next run. More info: https://energyplus.readthedocs.io/en/latest/datatransfer.html#datatransfer.DataExchange.request_variable
(RolloutWorker pid=3373334

Best Hyperparameters found:  Result(
  metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 4.097633175325393, 'cur_kl_coeff': 0.09999999999999999, 'cur_lr': 0.07996151037746047, 'total_loss': 9.94185263710022, 'policy_loss': 0.00467701552468352, 'vf_loss': 9.935975682067872, 'vf_explained_var': -1.0752677917480469e-08, 'kl': 0.011999143588970793, 'entropy': 6.392452412414551, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 126.464, 'num_grad_updates_lifetime': 23750.5, 'diff_num_grad_updates_vs_sampler_policy': 1249.5}}, 'num_env_steps_sampled': 400000, 'num_env_steps_trained': 400000, 'num_agent_steps_sampled': 400000, 'num_agent_steps_trained': 400000}, 'env_runners': {'episode_reward_max': 8139.454641578258, 'episode_reward_min': 7294.1674909568765, 'episode_reward_mean': 7648.403621653504, 'episode_len_mean': 4608.0, 'episode_media': {}, 'episodes_t

In [10]:
results[0].metrics

{'custom_metrics': {},
 'episode_media': {},
 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0,
     'grad_gnorm': 0.9375171548496651,
     'cur_kl_coeff': 0.20000000000000004,
     'cur_lr': 5.0000000000000016e-05,
     'total_loss': 10.015038082163821,
     'policy_loss': 0.014546937919572317,
     'vf_loss': 10.0,
     'vf_explained_var': 5.35864983835528e-06,
     'kl': 0.002455810879708283,
     'entropy': 1.3895884078036072,
     'entropy_coeff': 0.0},
    'model': {},
    'custom_metrics': {},
    'num_agent_steps_trained': 128.0,
    'num_grad_updates_lifetime': 465.5,
    'diff_num_grad_updates_vs_sampler_policy': 464.5}},
  'num_env_steps_sampled': 4000,
  'num_env_steps_trained': 4000,
  'num_agent_steps_sampled': 4000,
  'num_agent_steps_trained': 4000},
 'env_runners': {'episode_reward_max': nan,
  'episode_reward_min': nan,
  'episode_reward_mean': nan,
  'episode_len_mean': nan,
  'episode_media': {},
  'episodes_timesteps_total': 0,
  '

In [3]:
from ray.rllib.algorithms.ppo import PPO, PPOConfig
import itables as _itables_
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from controllables.core.tools.records import VariableRecords
from controllables.energyplus import Actuator, OutputVariable, OutputMeter

class PMVVariable(BaseVariable):
    def __init__(
        self, 
        tdb: BaseVariable,
        tr: BaseVariable,
        rh: BaseVariable,
        metab_rate=1.5, clothing=.5, pmv_limit=.5,
    ):
        self.tdb = tdb
        self.tr = tr
        self.rh = rh
        self._metab_rate = _numpy_.asarray(metab_rate)
        self._clothing = _numpy_.asarray(clothing)
        self._pmv_limit = _numpy_.asarray(pmv_limit)
    
    @property
    def value(self):
        res = pytc.models.pmv_ppd(
            tdb=self.tdb.value, 
            tr=self.tr.value, 
            # calculate relative air speed
            vr=pytc.utilities.v_relative(v=0.1, met=self._metab_rate), 
            rh=self.rh.value, 
            met=self._metab_rate, 
            # calculate dynamic clothing
            clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
            limit_inputs=False,
        )['pmv']

        return res

class PlottingCallbacks(DefaultCallbacks):
    def __init__(self):
        self.env_records = None

    def on_episode_start(self, *, episode, worker, **kwargs):
        env: CustomExternalEnv = worker.env
        system = env.system
        if self.env_records is None:
            tdb = system[OutputVariable.Ref('Zone Mean Air Temperature', '1FFIRSTFLOORWEST:OPENOFFICE')]
            tr =system[OutputVariable.Ref('Zone Mean Radiant Temperature', '1FFIRSTFLOORWEST:OPENOFFICE')]
            rh = system[OutputVariable.Ref('Zone Air Relative Humidity', '1FFIRSTFLOORWEST:OPENOFFICE')]
            pmv = PMVVariable(tdb=tdb, tr=tr, rh=rh)
            self.env_records = records = VariableRecords({
                '🕰️': system['wallclock:calendar'],
                #'🍩': env.agent.reward,
                'pmv': pmv,
                'AHU COOLING COIL':system[OutputVariable.Ref('Cooling Coil Total Cooling Rate', 'AIR LOOP AHU COOLING COIL')],
                'temp':system[OutputVariable.Ref('Zone Mean Air Temperature', '1FFIRSTFLOORWEST:OPENOFFICE')],
                'elec': system[OutputMeter.Ref('Electricity:HVAC')]
            })
            # display(
            #     records.plot.scatter(x='🕰️', y='pmv')
            #     .watch(records.events['change'] % 1_000)
            # )

    def on_episode_step(self, *, episode, **kwargs):
        self.env_records.poll()

    def on_episode_end(self, *, episode, **kwargs):
        df = self.env_records.dataframe()
        df.to_csv('tmp/records_single_fix_penalty.csv', index=False)
        display(_itables_.show(df))


In [4]:
config_eval = (
    get_config()
    .env_runners(
        num_env_runners=0,
        create_env_on_local_worker=True,
    )
    .evaluation(
        # evaluation_duration=1,
        # evaluation_duration_unit='episodes',
        #evaluation_interval=1,
        evaluation_num_env_runners=0,
    )
    .callbacks(PlottingCallbacks)
)

algo_eval = PPO(config_eval)
algo_eval.restore('/home/AD/user/ray_results/PPO_2024-10-30_06-17-02/PPO_CustomExternalEnv_956b5_00000_0_2024-10-30_06-17-04/checkpoint_000010')

/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:555: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationW

In [ ]:
for _ in range(2):
    algo_eval.evaluate()

/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/variables.py:345: RuntimeWarning: OutputVariable(OutputVariable.Ref(type='Zone Air Relative Humidity', key='0FGROUNDFLOOREAST1:OPENOFFICE')) requested while Kernel() is running; It may not be available until the next run. More info: https://energyplus.readthedocs.io/en/latest/datatransfer.html#datatransfer.DataExchange.request_variable
  _warnings_.warn(
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/variables.py:345: RuntimeWarning: OutputVariable(OutputVariable.Ref(type='Zone Mean Radiant Temperature', key='1FFIRSTFLOORWEST1:OPENOFFICE')) requested while Kernel() is running; It may not be available until the next run. More info: https://energyplus.readthedocs.io/en/latest/datatransfer.html#datatransfer.DataExchange.request_variable
  _warnings_.warn(
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus

None

None

In [ ]:
from controllables.core.tools.records import VariableRecords
import itables as _itables_

records = VariableRecords({
    '🕰️': simulator['wallclock:calendar'],
    'AHU COOLING COIL': 
        simulator[OutputVariable.Ref('Cooling Coil Total Cooling Rate', 'AIR LOOP AHU COOLING COIL')] ,
}).watch(simulator.events['timestep'])


In [ ]:
df = records.dataframe()
df.to_csv('datasave/data_baseline.csv', index=False, sep=';')
_itables_.show(df)

In [ ]:
records.plot.scatter(x='🕰️', y='AHU COOLING COIL').watch(records.events['change'])